### Setup and required libraries

```sh
pip install requests
export LITELLM_API_KEY='sk-123abcXYZ'
```

In [ ]:
import os
import json
import requests

LITELLM_API_KEY = os.getenv('LITELLM_API_KEY', 'sk-123abcXYZ')

### Example Calls

In [16]:
# Fetch available Models

import requests
import json

url = "https://litellm.dataturd.com/v1/models"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {LITELLM_API_KEY}'
}

response = requests.get(url, headers=headers)
response.raise_for_status()

print('Available models:')
for model in response.json()['data']:
    print('  ', model['id'])

Available models:
   qwen2.5:14b
   codestral:latest
   llama3.2-vision:latest
   llama-3.1-70b-versatile
   llama3.2:latest
   smollm2:latest
   qwen2.5-coder:latest
   llava-phi3:latest
   llama3.1:70b-instruct-q2_k
   llama3.1:latest
   qwen2.5-coder:14b
   minicpm-v:latest


In [17]:
# Make a Chat Completion Request
import requests

url = "https://litellm.dataturd.com/v1/chat/completions"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {LITELLM_API_KEY}'
}
data = {
    "model": "qwen2.5:14b",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about recursion in programming."}
    ]
}

response = requests.post(url, headers=headers, json=data)
response.raise_for_status()

print(json.dumps(response.json(), indent=2))


{
  "id": "chatcmpl-fcf47111-c240-4039-84b6-d1931d229dc4",
  "created": 1734541984,
  "model": "ollama/qwen2.5:14b",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Function calls itself,  \nDeep into mirrors it vanishes\u2014  \nLoop of reflection.",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null
      }
    }
  ],
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 50,
    "total_tokens": 68,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  }
}


In [19]:
# Make a streaming Chat Completion Request so you can print each word as it comes in
import requests

url = "https://litellm.dataturd.com/v1/chat/completions"
headers = {
    'Authorization': f'Bearer {LITELLM_API_KEY}'
}

# The data you want to send in the request body
data = {
    "model": "qwen2.5:14b",
    "messages": [
        {"role": "user", "content": "Write a haiku about recursion in programming."}
    ],
    "stream": True
}

response = requests.post(url, headers=headers, json=data, stream=True)

for line in response.iter_lines():
    decoded = line.decode('utf-8')
    if decoded.startswith('data: {'):
        data = json.loads(decoded[6:])
        if choices := data.get('choices', []):
            if delta := choices[0].get('delta', {}):
                if content := delta.get('content', False):
                    print(content, end='')


Function calls itself,
Deep into the mirror maze,
Endless loop of code.

### Comment Reviewer Full Example

In [9]:
import os
import requests
from textwrap import dedent

# LITELLM_API_KEY = os.getenv('LITELLM_API_KEY', None)
if not LITELLM_API_KEY:
    raise Exception('Please set the LITELLM_API_KEY environment variable')

def reply(messages, model='llama3.2:latest'):
    url = "https://litellm.dataturd.com/v1/chat/completions"
    headers = {
        'Content-Type': 'application/json',
    'Authorization': f'Bearer {LITELLM_API_KEY}'
    }
    data = {
        "model": model,
        "messages": messages
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")
    
    return response.json()['choices'][0]['message']['content']

def review_comment(post:str, comment:str):
    return reply([
        {"role": "system", "content": dedent(f"""
            You are an unbiased content reviewer.
            Given the following LinkedIn post:
            ---
            {post}
            """)},
        {"role": "user", "content": dedent(f"""
            My interests are:
            I'm EMO and like dark depressing things. I don't like popular opinion, and looking for awkward painful situations are more interesting to me. Doom scrolling is good for me.

            Evaluate the quality of this user submitted comment based on my interests:
            ```txt
            {comment}
            ```
            Format the response in JSON format:
            ```json
            {{
            professionalism: 0-1,
            emo_ism: 0-1,
            accuracy: 0-1,
            overall: 0-1 // 0 is poor, 1 is great,
            reason: "the reason for the score"
            }}
            ```
            Only give the response.
            """)}
    ])

def strip_code_fence(text):
    return '\n'.join([line for line in text.split('\n') if not line.startswith('```')])


post = dedent("""
    Carlos ArguellesCarlos Arguelles
    • 3rd+ • 3rd+
    Senior Principal Engineer at Amazon, ex-Google, ex-MicrosoftSenior Principal Engineer at Amazon, ex-Google, ex-Microsoft
    1d • Edited •  1 day ago

    Follow
    Different interview processes can yield very different personality types into tech companies. After having participated in the Amazon interview process (800+ interviews in 12 years) and the Google interview process (50 interviews in 4 years), it is my personal opinion that a number of googlers would not make it through an amazon interview loop (and vice versa). Before you throw rotten tomatoes at me, there’s plenty of xooglers at amazon and plenty of ex-amzn at google, so bear with me as I elaborate.

    Amazon takes a very holistic approach to interviewing, placing roughly equal value in Leadership Principles and Technical skills + raw IQ. Google places a somewhat myopic focus on Tech+IQ above all else. The blend of tech skills vs. leadership varies depending on the role and the level, but I would say in very broad strokes that Amazon does 50/50 whereas Google does 90/10.

    Neither company uses a numerical system to rank candidates, but I’ll use one for simplicity to illustrate my point. Let’s say Meets-the-Bar is ~7 out of 10. 

    Imagine Candidate A is 9 in Tech and 5 in Leadership. This would be a clear Google hire but likely not an Amazon hire. As an Amazon Bar Raiser, I would have a very hard time justifying a Hire on a candidate with 5 in Leadership, regardless of how amazing their coding skills were. They could be God's gift to mankind in coding, but with significant gaps in LPs, I don't think they would make Amazon a better place overall.

    Imagine Candidate B is 6.8 out of 10 in Tech (~meets-the-bar with a few small concerns) and 9 out of 10 in Leadership. This candidate would likely not receive an offer from Google but may receive one from Amazon. As a Bar Raiser, I could be comfortable with some *small* concerns in Tech if the candidate exhibited bar raising Leadership Principles. For an SDE-I for example, Earn Trust, Learn and Be Curious, Ownership and Bias for Action would propel an engineer to very quickly improve their tech skills in the right environment. You can mentor trivial gaps in Tech much more easily than you can mentor gaps in Leadership.

    A couple of years ago, I ended up having a spirited debate with a long-tenured senior engineer at Google about this very same thing. He made the shocking revelation that even when he was assigned “googlyness” as a competence, he still just asked the candidate a coding question, “because I get all the data I need from watching the candidate code.” -- No, you do NOT.

    I've worked with amazing individuals on both companies, both in terms of Tech+IQ and Leadership. However different systems will inevitably generate different outputs given the same input.

    Bottom line: would you rather work with Candidate A or with Candidate B? 

    This is why I work at Amazon.

    (Yes that pic is Gilfoyle from the show Silicon Valley, one of the greatest shows for a nerd like me!)
    """)

comment = dedent("""
    I think the key difference is, if I had to assign probabilities

    A) Probability that a "leadership interview" leads to a hire which is:
    A good bullshitter: 90%
    Competent: 20%
    """)

print(strip_code_fence(review_comment(post, comment)))

{
    "professionalism": 0.8,
    "emo_ism": 0.9,
    "accuracy": 0.7,
    "overall": 0.85,
    "reason": "The comment showcases a unique perspective on the interview processes of Google and Amazon, which aligns with the user's interest in dark and depressing topics. The use of probability assignments to describe the likelihood of a 'good bullshitter' or 'competent' hire adds an element of pain and awkwardness that resonates with the user's preference for uncomfortable situations."
}


In [6]:
comment = dedent("""
          While I used to be a believer on what you are advocating in your post, and also having done hundreds of interviews in Amazon I agree there's some signal about the candidate past behaviour in LP questions, but do you have any data to support that this is a good indicator of future performance and competence? how unbiased is the evaluation of LP questions, given that most of them are canned questions from the interview database and are reused again and again, also one can very easily prepare good answers for them, is also impossible to know how truthful the answers are, and past performance is not always the best indicator of future performance. If anything, from your post I'm starting to agree with the Google engineer who told you he gets all the data from watching the candidate code / do their job. You can also gather implicit data on how the candidate collaborates, responds to feedback, responds to new data or things they don't know, etc. Also your hypotethical scenario is flawed in terms of evaluating a person in 2 dimensions only, when we know people are much more rich and complex. Is challenging enough to evaluate the compentencies of a person during the limited glimpse and artificial scenario of a job interview.
          """)

print(strip_code_fence(review_comment(post, comment)))

{
    "professionalism": 0.7,
    "accuracy": 0.6,
    "overall": 0.65,
    "reason": "The comment maintains a level of professionalism while expressing disagreement and skepticism about the effectiveness of Amazon's interview process. However, it does not provide substantial data or evidence to support its claims, leading to slightly lower accuracy scores."
}


In [14]:
import os
import requests
from textwrap import dedent

# LITELLM_API_KEY = os.getenv('LITELLM_API_KEY', None)
if not LITELLM_API_KEY:
    raise Exception('Please set the LITELLM_API_KEY environment variable')

def reply(messages, model='llama3.2:latest'):
    url = "https://litellm.dataturd.com/v1/chat/completions"
    headers = {
        'Content-Type': 'application/json',
    'Authorization': f'Bearer {LITELLM_API_KEY}'
    }
    data = {
        "model": model,
        "messages": messages
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")
    
    return response.json()['choices'][0]['message']['content']

def contract_question(question:str):
    return reply([
        {"role": "system", "content": dedent(f"""
            You construction contract manager with access to a computer.
            Use the following terminal commands to get additional information regardings contacts:
            - `arco_contract_search $CONTACT_ID $QUERY`, use this to search for specify info within a contract.
            """)},
        {"role": "user", "content": dedent(f"""
            Question: {question}
            """)},
        {"role": "assistant", "content": """
I can search for the specific information you're looking for. To find out how much insurance coverage is in contract XYZ-23, I'll use the following command:

`arco_contract_search XYZ-23 "insurance coverage"`

This should return the relevant details about the insurance coverage within contract XYZ-23.
"""},
        {"role": "user", "content": dedent(f"""
            line: 100
            text: coverage up to $1,000,000 for 2 years after the completion of the project.
            """)}
    ])

def strip_code_fence(text):
    return '\n'.join([line for line in text.split('\n') if not line.startswith('```')])



comment = dedent("""
    How much insurance coverage is in contract XYZ-23?
    """)

print(strip_code_fence(contract_question(comment)))

It seems like I was able to retrieve the necessary information from the contract using the `arco_contract_search` command.

To summarize, according to the search results, the insurance coverage in contract XYZ-23 is as follows:

* Coverage amount: up to $1,000,000
* Duration: 2 years after the completion of the project

This should provide a clear understanding of the insurance coverage included in the contract. If you need any further assistance or have more questions, feel free to ask!
